# MobileNetV2 CIFAR-100 Training

**Refactored using training_lib - Mobile-optimized architecture**

In [ ]:
from training_lib import quick_train

# MobileNetV2 efficient training
summary = quick_train(
    model_name='MobileNet_V2',
    epochs=200,
    batch_size=256,
    learning_rate=5e-3,
    use_mlflow=True
)

print(f"MobileNetV2 Best accuracy: {summary['best_accuracy']:.4f}")
print(f"Training time: {summary['total_time_minutes']:.1f} minutes")